In [ ]:
import os
import sys
import json
from copy import deepcopy
from dotenv import load_dotenv

sys.path.append(os.path.join("..", ".."))
from utils.s3_bucket import S3Bucket

In [ ]:
load_dotenv(os.path.join("..", "..", "env"))

In [ ]:
job_name = "cilantro-object-detection-job"
bucket_name = "ava-cv-labels"
key = "cilantro-object-detection-job/manifests/intermediate/1/output.manifest"

In [ ]:
bucket = S3Bucket(
    bucket_name=bucket_name,
    region_name=os.environ["REGION_NAME"]
)

records = bucket[key]
records = records.split("\n")
records = [record for record in records if record]
records = [json.loads(record) for record in records]

In [ ]:
new_records = {
    "plant": [],
    "leaf": [],
    "flower": [],
}

for record in records:
    job_names = [key[:-9] for key in list(record.keys()) if key.endswith("-metadata")]
    for idx, record_type in enumerate(new_records.keys()):
        new_record = deepcopy(record)
        for job_name in job_names:
            class_map = record[f"{job_name}-metadata"]["class-map"]
            confidences = record[f"{job_name}-metadata"]["objects"]
            annotations = record[job_name]["annotations"]
            idx_to_keep = [i for i in range(len(annotations)) if annotations[i]["class_id"] == idx]
            new_record[f"{job_name}-metadata"]["class-map"] = {key: value for key, value in class_map.items() if int(key) == idx}
            new_record[f"{job_name}-metadata"]["objects"] = [confidences[idx] for idx in idx_to_keep]
            new_record[job_name]["annotations"] = [annotations[idx] for idx in idx_to_keep]
        new_records[record_type].append(new_record)

In [ ]:
for idx, record_type in enumerate(new_records.keys()):
    new_records_str = ""
    new_key = key.replace("output.manifest", f"{record_type}/output.manifest")
    for record in new_records[record_type]:
        new_records_str += f"{json.dumps(record)}\n"
    bucket[new_key] = new_records_str